# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps

import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
input_data_file = "../output_data/city_data.csv"
city_data = pd.read_csv(input_data_file)
city_data = city_data.dropna()
display(city_data)

,Unnamed: 0,City ID,City Name,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,0,4032243,Vaini,-21.2000,-175.2000,25.64,85,76,5.33
1,1,2206939,Bluff,-46.6000,168.3333,9.89,90,100,10.09
2,2,1006984,East London,-33.0153,27.9116,14.44,82,67,5.93
3,3,6111862,Port Hardy,50.6996,-127.4199,11.43,77,99,2.62
4,4,2074865,Carnarvon,-24.8667,113.6333,22.04,68,40,6.17
...,...,...,...,...,...,...,...,...,...
539,539,1810979,Fuling,29.7022,107.3919,27.23,59,99,1.21
540,540,1732738,Kota Tinggi,1.7381,103.8999,33.65,52,64,1.84
541,541,2090021,Namatanai,-3.6667,152.4333,28.63,74,60,3.15
542,542,2012530,Zhigansk,66.7697,123.3711,15.71,28,2,3.89


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = city_data[["Latitude", "Longitude"]].astype(float)
weights = city_data['Humidity']

heat_layer = gmaps.heatmap_layer(locations, weights = weights)
fig = gmaps.figure()
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [85]:
hotel_df = city_data[city_data['Temperature']<5]
hotel_df = hotel_df[hotel_df['Cloudiness']==100]
hotel_df = hotel_df.reset_index()
display(hotel_df)


,index,Unnamed: 0,City ID,City Name,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,13,13,3421765,Nanortalik,60.1432,-45.2371,1.48,99,100,3.40
1,45,45,780687,Berlevåg,70.8578,29.0864,4.98,87,100,12.55
2,56,56,3424607,Tasiilaq,65.6145,-37.6368,3.09,98,100,8.31
3,123,123,5954718,Flin Flon,54.7682,-101.8650,4.91,85,100,4.94
4,149,149,6167817,Torbay,47.6666,-52.7314,4.92,84,100,1.34
5,179,179,3418910,Upernavik,72.7868,-56.1549,-0.70,95,100,7.14
6,208,208,3421719,Narsaq,60.9167,-46.0500,3.12,98,100,1.35
7,219,219,1503037,Kodinsk,58.6881,99.1844,4.54,83,100,3.31
8,261,261,1510689,Baykit,61.6700,96.3700,4.84,84,100,3.30
9,277,277,2025256,Chumikan,54.7183,135.3150,2.31,91,100,4.86


index                  13
Unnamed: 0             13
City ID           3421765
City Name      Nanortalik
Latitude          60.1432
Longitude        -45.2371
Temperature          1.48
Humidity               99
Cloudiness            100
Wind Speed            3.4
Name: 0, dtype: object

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [114]:
hotel_list = pd.DataFrame()

for row in range(len(hotel_df)):

    Lat = (hotel_df.iloc[row]['Latitude'])
    Long = (hotel_df.iloc[row]['Longitude'])    

    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?query=parameters&location={Lat},{Long}&radius=5000&type=lodging&key={g_key}"
    response = requests.get(url).json()
    try:
        hotel = [response['results'][0]['name'],
        response['results'][0]['plus_code']['compound_code']]
        
        #pulls hotel, city and country information from the API call and places it into a list
        hotel[1] = hotel[1].split(' ', 1)[1]
        hotel[1] = hotel[1].replace(" ", "")
        hotel.append(hotel[1].split(',')[-1])
        hotel[1] = hotel[1].split(',')[0]
        
        hotel.append(Lat)
        hotel.append(Long)
        
        hotel_list = hotel_list.append([hotel])
        
        
        print(hotel)
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

    
#end For
hotel_list = hotel_list.reset_index()
hotel_list = hotel_list.drop('index', 1)
hotel_list.columns = ['Hotel Name', 'City', 'Country','Latitude', 'Longitude']
display(hotel_list)


['Hotel Kap Farvel', 'Nanortalik', 'Greenland', 60.1432, -45.2371]
['Kvitbrakka', 'Bearalváhki', 'Norway', 70.8578, 29.0864]
['The Red House', 'Tasiilaq', 'Greenland', 65.6145, -37.6368]
['Victoria Inn Flin Flon', 'FlinFlon', 'Canada', 54.7682, -101.865]
['Gallows Cove', 'Torbay', 'Canada', 47.6666, -52.7314]
["Gina's Guesthouse - Upernavik", 'Upernavik', 'Greenland', 72.7868, -56.1549]
['Narsaq Kayak Hostel', 'Narsaq', 'Greenland', 60.9167, -46.05]
Missing field/result... skipping.
Missing field/result... skipping.
['Fortuna', 'Chumikan', 'Russia', 54.7183, 135.315]
['Auberge Madeli', 'Cap-aux-Meules', 'Canada', 47.3821, -61.8597]
['Marystown Hotel & Convention Centre', 'Marystown', 'Canada', 47.1666, -55.1483]
['Magadan hotel', 'Magadan', 'Russia', 59.5667, 150.8]


,Hotel Name,City Name,Country,Latitude,Longitude
0,Hotel Kap Farvel,Nanortalik,Greenland,60.1432,-45.2371
1,Kvitbrakka,Bearalváhki,Norway,70.8578,29.0864
2,The Red House,Tasiilaq,Greenland,65.6145,-37.6368
3,Victoria Inn Flin Flon,FlinFlon,Canada,54.7682,-101.8650
4,Gallows Cove,Torbay,Canada,47.6666,-52.7314
5,Gina's Guesthouse - Upernavik,Upernavik,Greenland,72.7868,-56.1549
6,Narsaq Kayak Hostel,Narsaq,Greenland,60.9167,-46.0500
7,Fortuna,Chumikan,Russia,54.7183,135.3150
8,Auberge Madeli,Cap-aux-Meules,Canada,47.3821,-61.8597
9,Marystown Hotel & Convention Centre,Marystown,Canada,47.1666,-55.1483


In [116]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [ ]:
# Add marker layer ontop of heat map


# Display figure


'Greenland'